In [3]:
import os
import cv2
import dlib
import multiprocessing

# Recortar rosto do animal e salvar a imagem

In [4]:
def process_image(img_path, input_dir, output_dir):
    try:
        detector = dlib.cnn_face_detection_model_v1('dlib_models/dogHeadDetector.dat')
        filename, ext = os.path.splitext(os.path.basename(img_path))
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        dets = detector(img, upsample_num_times=1)
        
        for i, d in enumerate(dets):

            x1, y1 = d.rect.left(), d.rect.top()
            x2, y2 = d.rect.right(), d.rect.bottom()
            
            roi = img[y1:y2, x1:x2]
            
            output_subdir = os.path.join(output_dir, os.path.relpath(os.path.dirname(img_path), input_dir))
            os.makedirs(output_subdir, exist_ok=True)
            output_filename = f"{filename}_face{i+1}.jpg"
            output_path = os.path.join(output_subdir, output_filename)
            cv2.imwrite(output_path, cv2.cvtColor(roi, cv2.COLOR_RGB2BGR))
    except Exception as e:
        print(f"Erro na imagem {img_path}: {str(e)}")

def process_images_in_directory(input_dir, output_dir):
    with multiprocessing.Pool() as pool:
        image_paths = []
        for root, dirs, files in os.walk(input_dir):
            for file in files:
                if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                    img_path = os.path.join(root, file)
                    image_paths.append(img_path)

        pool.starmap(process_image, [(img_path, input_dir, output_dir) for img_path in image_paths])

In [ ]:
# Exemplo de uso:
input_directory = "images"
output_directory = "images_processed"
process_images_in_directory(input_directory, output_directory)

## Train e Test

In [11]:
import os
import shutil
import numpy as np
from sklearn.model_selection import train_test_split
from shutil import copyfile

In [6]:
src_dir = 'images_processed'
train_dir = 'images_processed/train'
test_dir = 'images_processed/test'

In [12]:
def clean_old_test_train(path):
    if os.path.exists(path):
        shutil.rmtree(path)  # Use rmtree para remover um diretório e seu conteúdo
        print(f"Diretório '{path}' removido com sucesso.")
    else:
        print(f"Diretório '{dpath}' não existe.")

In [13]:
clean_old_test_train(train_dir)
clean_old_test_train(test_dir)

Diretório 'images_processed/train' removido com sucesso.
Diretório 'images_processed/test' removido com sucesso.


In [14]:
labels = os.listdir(src_dir)  # Isso irá obter todas as labels a partir dos subdiretórios

for label in labels:
    src_label_dir = os.path.join(src_dir, label)
    train_label_dir = os.path.join(train_dir, label)
    test_label_dir = os.path.join(test_dir, label)
    
    os.makedirs(train_label_dir, exist_ok=True)
    os.makedirs(test_label_dir, exist_ok=True)
    
    files = os.listdir(src_label_dir)
    train_files, test_files = train_test_split(files, test_size=0.2, random_state=42)
    
    for file in train_files:
        src_file = os.path.join(src_label_dir, file)
        dst_file = os.path.join(train_label_dir, file)
        copyfile(src_file, dst_file)
    
    for file in test_files:
        src_file = os.path.join(src_label_dir, file)
        dst_file = os.path.join(test_label_dir, file)
        copyfile(src_file, dst_file)

## Data augmentation

In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2023-09-24 01:51:29.594593: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-24 01:51:29.595899: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-24 01:51:29.624373: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-24 01:51:29.624677: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-24 01:51:30.243406: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [16]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

In [17]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(150, 150),
                                                    batch_size=20,
                                                    class_mode='categorical')

Found 4851 images belonging to 4 classes.


In [18]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [19]:
test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(150, 150),
                                                  batch_size=20,
                                                  class_mode='categorical')

Found 1215 images belonging to 4 classes.


## CNN

In [20]:
import tensorflow as tf
from keras.callbacks import EarlyStopping

In [21]:
def create_and_train_model(train_generator, test_generator, num_epochs=30):
    
    #Callback EarlyStopping
    early_stopping = EarlyStopping(monitor='val_loss',
                                   patience=10,
                                   verbose=1,
                                   restore_best_weights=True)

    #Definição da rede
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(4, activation='softmax')
    ])

    model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(lr=1e-5),
                  metrics=['accuracy'])

    history = model.fit(train_generator,
                        steps_per_epoch=100,
                        epochs=num_epochs,
                        validation_data=test_generator,
                        validation_steps=50,
                        callbacks=[early_stopping])
    
    return model, history

In [22]:
model, history = create_and_train_model(train_generator, test_generator, num_epochs=50)

2023-09-24 01:51:34.791715: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-24 01:51:34.792059: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/50
100/100 [==============================] - 19s 182ms/step - loss: 1.1513 - accuracy: 0.3910 - val_loss: 1.0984 - val_accuracy: 0.4280
Epoch 2/50
100/100 [==============================] - 19s 185ms/step - loss: 1.0988 - accuracy: 0.4320 - val_loss: 1.1041 - val_accuracy: 0.4230
Epoch 3/50
100/100 [==============================] - 19s 192ms/step - loss: 1.1067 - accuracy: 0.3887 - val_loss: 1.1194 - val_accuracy: 0.3900
Epoch 4/50
100/100 [==============================] - 19s 188ms/step - loss: 1.1060 - accuracy: 0.4199 - val_loss: 1.1084 - val_accuracy: 0.4280
Epoch 5/50
100/100 [==============================] - 19s 192ms/step - loss: 1.1036 - accuracy: 0.4093 - val_loss: 1.1167 - val_accuracy: 0.4180
Epoch 6/50
100/100 [==============================] - 20s 197ms/step - loss: 1.0939 - accuracy: 0.4145 - val_loss: 1.0918 - val_accuracy: 0.4340
Epoch 7/50
100/100 [==============================] - 19s 194ms/step - loss: 1.1152 - accuracy: 0.4063 - val_loss: 1.0914 - val_ac

In [23]:
test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print('\nTest accuracy:', test_acc)

61/61 - 3s - loss: 1.0967 - accuracy: 0.4239 - 3s/epoch - 47ms/step

Test accuracy: 0.4238682985305786


## DenseNet

In [28]:
from tensorflow.keras.applications import DenseNet121

In [29]:
def create_and_train_model_densenet(train_generator, test_generator, num_epochs=30):
    
    #Callback EarlyStopping
    early_stopping = EarlyStopping(monitor='val_loss',
                                   patience=10,
                                   verbose=1,
                                   restore_best_weights=True)
    
    densenet_base = tf.keras.applications.DenseNet121(weights='imagenet',
                                                      include_top=False,
                                                      input_shape=(150, 150, 3))
    
    # Congelar as camadas do DenseNet121
    densenet_base.trainable = False

    model_densenet = tf.keras.models.Sequential([
        densenet_base,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(4, activation='softmax')
    ])
    
    model_densenet.compile(loss='categorical_crossentropy',
                           optimizer=tf.keras.optimizers.Adam(lr=1e-5),
                           metrics=['accuracy'])

    history_densenet = model_densenet.fit(train_generator,
                                          steps_per_epoch=100,
                                          epochs=30,
                                          validation_data=test_generator,
                                          validation_steps=50,
                                          callbacks=[early_stopping])
    
    return model_densenet, history_densenet

In [30]:
model_densenet, history_densenet = create_and_train_model_densenet(train_generator, test_generator, num_epochs=50)

Epoch 1/30
100/100 [==============================] - 48s 442ms/step - loss: 3.0099 - accuracy: 0.4880 - val_loss: 1.0313 - val_accuracy: 0.5990
Epoch 2/30
100/100 [==============================] - 43s 433ms/step - loss: 1.0559 - accuracy: 0.5530 - val_loss: 0.9632 - val_accuracy: 0.5980
Epoch 3/30
100/100 [==============================] - 43s 433ms/step - loss: 0.9466 - accuracy: 0.5846 - val_loss: 0.9502 - val_accuracy: 0.6130
Epoch 4/30
100/100 [==============================] - 44s 437ms/step - loss: 0.9233 - accuracy: 0.5985 - val_loss: 0.8887 - val_accuracy: 0.6390
Epoch 5/30
100/100 [==============================] - 44s 437ms/step - loss: 0.9371 - accuracy: 0.6110 - val_loss: 0.8723 - val_accuracy: 0.6360
Epoch 6/30
100/100 [==============================] - 44s 436ms/step - loss: 0.8994 - accuracy: 0.6113 - val_loss: 0.8845 - val_accuracy: 0.6290
Epoch 7/30
100/100 [==============================] - 44s 438ms/step - loss: 0.8782 - accuracy: 0.6070 - val_loss: 1.1042 - val_ac

In [31]:
test_loss_densenet, test_acc_densenet = model_densenet.evaluate(test_generator, verbose=2)
print('\nTest accuracy:', test_acc_densenet)

61/61 - 16s - loss: 0.8173 - accuracy: 0.6733 - 16s/epoch - 259ms/step

Test accuracy: 0.6732510328292847


## EffiicientNet

In [32]:
from tensorflow.keras.applications import EfficientNetB0

In [33]:
def create_and_train_model_efficientnet(train_generator, test_generator, num_epochs=30):
    
    # Callback EarlyStopping
    early_stopping = EarlyStopping(monitor='val_loss',
                                   patience=10,
                                   verbose=1,
                                   restore_best_weights=True)
    
    efficientnet_base = EfficientNetB0(weights='imagenet',
                                       include_top=False,
                                       input_shape=(150, 150, 3))
    
    # Congelar as camadas do EfficientNetB0 para evitar que sejam treinadas
    efficientnet_base.trainable = False

    model_efficientnet = tf.keras.models.Sequential([
        efficientnet_base,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(4, activation='softmax')
    ])
    
    model_efficientnet.compile(loss='categorical_crossentropy',
                               optimizer=tf.keras.optimizers.Adam(lr=1e-5),
                               metrics=['accuracy'])

    history_efficientnet = model_efficientnet.fit(train_generator,
                                                  steps_per_epoch=100,
                                                  epochs=num_epochs,
                                                  validation_data=test_generator,
                                                  validation_steps=50,
                                                  callbacks=[early_stopping])
    
    return model_efficientnet, history_efficientnet

In [34]:
model_efcnet, history_efcnet = create_and_train_model_efficientnet(train_generator, test_generator, num_epochs=50)

Epoch 1/50
100/100 [==============================] - 27s 231ms/step - loss: 4.8206 - accuracy: 0.3702 - val_loss: 3.3875 - val_accuracy: 0.4260
Epoch 2/50
100/100 [==============================] - 22s 220ms/step - loss: 3.3088 - accuracy: 0.3646 - val_loss: 3.7003 - val_accuracy: 0.4210
Epoch 3/50
100/100 [==============================] - 22s 221ms/step - loss: 1.8791 - accuracy: 0.3913 - val_loss: 1.7857 - val_accuracy: 0.1820
Epoch 4/50
100/100 [==============================] - 22s 222ms/step - loss: 1.9088 - accuracy: 0.3822 - val_loss: 2.2748 - val_accuracy: 0.1810
Epoch 5/50
100/100 [==============================] - 22s 224ms/step - loss: 1.7560 - accuracy: 0.3596 - val_loss: 1.3180 - val_accuracy: 0.4190
Epoch 6/50
100/100 [==============================] - 22s 222ms/step - loss: 1.5989 - accuracy: 0.3757 - val_loss: 1.4944 - val_accuracy: 0.1650
Epoch 7/50
100/100 [==============================] - 22s 222ms/step - loss: 1.4403 - accuracy: 0.3855 - val_loss: 1.6241 - val_ac

In [35]:
test_loss_efcnet, test_acc_efcnet = model_efcnet.evaluate(test_generator, verbose=2)
print('\nTest accuracy:', test_acc_efcnet)

61/61 - 6s - loss: 1.0969 - accuracy: 0.4239 - 6s/epoch - 99ms/step

Test accuracy: 0.4238682985305786


## Salvar modelos

In [36]:
def save_model(model, filename):
    model.save('cnn_models/'+filename)

In [37]:
save_model(model, 'model.h5')
save_model(model_densenet, 'model_densenet.h5')
save_model(model_efcnet, 'model_efcnet.h5')

/home/leonardocapellaro/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Predizer imagens

In [38]:
def process_image_predict(img_path, target_size=(150, 150)):
    try:
        detector = dlib.cnn_face_detection_model_v1('dlib_models/dogHeadDetector.dat')
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        dets = detector(img, upsample_num_times=1)

        # Se nenhuma detecção de face for encontrada, retorne uma imagem em branco
        if len(dets) == 0:
            print("Não foram encontrados rostos na imagem")
            return np.zeros((target_size[0], target_size[1], 3), dtype=np.uint8)

        # Desenhe retângulos nas detecções
        for i, d in enumerate(dets):
            x1, y1 = d.rect.left(), d.rect.top()
            x2, y2 = d.rect.right(), d.rect.bottom()
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Converta a imagem resultante de volta para BGR para salvar ou exibir
        img_result = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        # Redimensione a imagem resultante para o tamanho desejado
        img_result = cv2.resize(img_result, target_size)

        return img_result

    except Exception as e:
        print(f"Erro na imagem {img_path}: {str(e)}")

In [39]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

def image_class_predict(model, image_path):
    classes = ["angry", "happy", "sad"]
    # Carregue e pré-processe a imagem de entrada
    img = process_image_predict(image_path)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalização (se necessário)

    # Faça a previsão
    predictions = model.predict(img_array)

    # Obtenha a classe prevista
    class_index = np.argmax(predictions)

    return class_index, classes[class_index]

## Resultado CNN Simples

In [40]:
imagem = 'example.jpeg'
classe_prevista_index, classe_prevista = image_class_predict(model, imagem)

print(f'A imagem é da classe: {classe_prevista}')

1/1 [==============================] - 0s 81ms/step
A imagem é da classe: happy


## Resultado Densenet

In [42]:
imagem = 'example.jpeg'
classe_prevista_index, classe_prevista = image_class_predict(model_densenet, imagem)

print(f'A imagem é da classe: {classe_prevista}')

1/1 [==============================] - 1s 1s/step
A imagem é da classe: happy


## Resultado EFC Net

In [43]:
imagem = 'example.jpeg'
classe_prevista_index, classe_prevista = image_class_predict(model_efcnet, imagem)

print(f'A imagem é da classe: {classe_prevista}')

1/1 [==============================] - 1s 824ms/step
A imagem é da classe: happy
